# Wide and Deep on TensorFlow (notebook style)

Copyright 2016 Google Inc. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
 http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

This notebook uses the tf.learn API in TensorFlow to answer a yes/no question. This is called a binary classification problem: Given census data about a person such as age, gender, education and occupation (the features), we will try to predict whether or not the person earns more than 50,000 dollars a year (the target label). 

Given an individual's information our model will output a number between 0 and 1, which can be interpreted as the model's certainty that the individual has an annual income of over 50,000 dollars, (1=True, 0=False)


# Imports and constants
First we'll import our libraries and set up some strings for column names. We also print out the version of TensorFlow we are running.

In [75]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR) # Set to INFO for tracking training, default is WARN 

print("Using TensorFlow version %s" % (tf.__version__))

CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation", 
                       "relationship", "race", "gender", "native_country"]

COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income_bracket"]



Using TensorFlow version 1.0.1


# Input file parsing

This section puts the file into a `Reader` which reads from the file one batch at a time. 

We set up the Tensors to be a dictionary of features mapping from their string name to the tensor value.

Note that the `_input_fn()` function is wrapped, enabling it to be used for different files.

NOTE: This reads from the input file directly via TensorFlow, rather than using an intermediate tool such as pandas to load the entire dataset into memory first. This is done to enable the system to scale to large inputs.

## More about input functions

The input function is how we will feed the input data into the model during training and evaluation. 
The structure that must be returned is a pair, where the first element is a dict of the column names (features) mapped to a tensor of values, and the 2nd element is a tensor of values representing the answers (labels). Recall that a tensor is just a general term for an n-dimensional array.

This could be represented as: `map(column_name => [Tensor of values]) , [Tensor of labels])`

More concretely, for this particular dataset, something like this:

    { 
      'age':            [ 39, 50, 38, 53, 28, … ], 
      'marital_status': [ 'Married-civ-spouse', 'Never-married', 'Widowed', 'Widowed' … ],
       ...
      'gender':           ['Male', 'Female', 'Male', 'Male', 'Female',, … ], 
    } , 
    [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
    
Additionally, we define which columns of the input data we will treat as categorical vs continuous, using the global `CATEGORICAL_COLUMNS`.

You can try different values for `BATCH_SIZE` to see how they impact your results

In [71]:
BATCH_SIZE = 40

def generate_input_fn(filename, batch_size=BATCH_SIZE):
    def _input_fn():
        filename_queue = tf.train.string_input_producer([filename])
        reader = tf.TextLineReader()
        # Reads out batch_size number of lines
        key, value = reader.read_up_to(filename_queue, num_records=batch_size)

        # record_defaults should match the datatypes of each respective column.
        record_defaults = [[0], [" "], [0], [" "], [0],
                           [" "], [" "], [" "], [" "], [" "],
                           [0], [0], [0], [" "], [" "]]
        # Decode CSV data that was just read out. 
        columns = tf.decode_csv(
            value, record_defaults=record_defaults)

        # features is a dictionary that maps from column names to tensors of the data.
        # income_bracket is the last column of the data. Note that this is NOT a dict.
#         features, income_bracket = dict(zip(COLUMNS, columns[:-1])), columns[-1]
        features = dict(zip(COLUMNS, columns))
        income_bracket = features.pop('income_bracket')
    
        # remove the fnlwgt key, which is not used
        features.pop('fnlwgt', 'fnlwgt key not found')

        # Sparse categorical features must be represented with an additional dimension. 
        # There is no additional work needed for the Continuous columns; they are the unaltered columns.
        # See docs for tf.SparseTensor for more info
        for feature_name in CATEGORICAL_COLUMNS:
            # Requires tensorflow >= 0.12
            features[feature_name] = tf.expand_dims(features[feature_name], -1)

        # Convert ">50K" to 1, and "<=50K" to 0
        labels = tf.to_int32(tf.equal(income_bracket, " >50K"))

        return features, labels

    return _input_fn

print('input function configured')

input function configured


# Create Feature Columns
This section configures the model with the information about the model. There are many parameters here to experiment with to see how they affect the accuracy.

This is the bulk of the time and energy that is often spent on making a machine learning model work, called *feature selection* or *feature engineering*. We choose the features (columns) we will use for training, and apply any additional transformations to them as needed. 

### Sparse Columns
First we build the sparse columns.

Use `sparse_column_with_keys()` for columns that we know all possible values for.

Use `sparse_column_with_hash_bucket()` for columns that we want the the library to automatically map values for us.

In [3]:
# Sparse base columns.
gender = tf.contrib.layers.sparse_column_with_keys(column_name="gender",
                                                 keys=["female", "male"])
race = tf.contrib.layers.sparse_column_with_keys(column_name="race",
                                               keys=["Amer-Indian-Eskimo",
                                                     "Asian-Pac-Islander",
                                                     "Black", "Other",
                                                     "White"])

education = tf.contrib.layers.sparse_column_with_hash_bucket(
  "education", hash_bucket_size=1000)
marital_status = tf.contrib.layers.sparse_column_with_hash_bucket(
  "marital_status", hash_bucket_size=100)
relationship = tf.contrib.layers.sparse_column_with_hash_bucket(
  "relationship", hash_bucket_size=100)
workclass = tf.contrib.layers.sparse_column_with_hash_bucket(
  "workclass", hash_bucket_size=100)
occupation = tf.contrib.layers.sparse_column_with_hash_bucket(
  "occupation", hash_bucket_size=1000)
native_country = tf.contrib.layers.sparse_column_with_hash_bucket(
  "native_country", hash_bucket_size=1000)

print('Sparse columns configured')

Sparse columns configured


### Continuous columns
Second, configure the real-valued columns using `real_valued_column()`. 

In [4]:
# Continuous base columns.
age = tf.contrib.layers.real_valued_column("age")
education_num = tf.contrib.layers.real_valued_column("education_num")
capital_gain = tf.contrib.layers.real_valued_column("capital_gain")
capital_loss = tf.contrib.layers.real_valued_column("capital_loss")
hours_per_week = tf.contrib.layers.real_valued_column("hours_per_week")

print('continuous columns configured')

continuous columns configured


### Transformations
Now for the interesting stuff. We will employ a couple of techniques to get even more out of the data.
 
* **bucketizing** turns what would have otherwise been a continuous feature into a categorical one. 
* **feature crossing** allows us to compute a model weight for specific pairings across columns, rather than learning them as independently. This essentially encodes related columns together, for situations where having 2 (or more) columns being certain values is meaningful. 

Only categorical features can be crossed. This is one reason why age has been bucketized.

For example, crossing education and occupation would enable the model to learn about: 

    education="Bachelors" AND occupation="Exec-managerial"

or perhaps 

    education="Bachelors" AND occupation="Craft-repair"

We do a few combined features (feature crosses) here. 

Add your own, based on your intuitions about the dataset, to try to improve on the model!

In [5]:
# Transformations.
age_buckets = tf.contrib.layers.bucketized_column(age,
            boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])
education_occupation = tf.contrib.layers.crossed_column([education, occupation], hash_bucket_size=int(1e4))
age_race_occupation = tf.contrib.layers.crossed_column( [age_buckets, race, occupation], hash_bucket_size=int(1e6))
country_occupation = tf.contrib.layers.crossed_column([native_country, occupation], hash_bucket_size=int(1e4))

print('Transformations complete')

Transformations complete


### Group feature columns into 2 objects

The wide columns are the sparse, categorical columns that we specified, as well as our hashed, bucket, and feature crossed columns. 

The deep columns are composed of embedded categorical columns along with the continuous real-valued columns. **Column embeddings** transform a sparse, categorical tensor into a low-dimensional and dense real-valued vector. The embedding values are also trained along with the rest of the model. For more information about embeddings, see the TensorFlow tutorial on [Vector Representations Words](https://www.tensorflow.org/tutorials/word2vec/), or [Word Embedding](https://en.wikipedia.org/wiki/Word_embedding) on Wikipedia.

The higher the dimension of the embedding is, the more degrees of freedom the model will have to learn the representations of the features. We are starting with an 8-dimension embedding for simplicity, but later you can come back and increase the dimensionality if you wish.



In [6]:
# Wide columns and deep columns.
wide_columns = [gender, native_country,
      education, occupation, workclass,
      marital_status, relationship,
      age_buckets, education_occupation,
      age_race_occupation, country_occupation]

deep_columns = [
  tf.contrib.layers.embedding_column(workclass, dimension=8),
  tf.contrib.layers.embedding_column(education, dimension=8),
  tf.contrib.layers.embedding_column(marital_status, dimension=8),
  tf.contrib.layers.embedding_column(gender, dimension=8),
  tf.contrib.layers.embedding_column(relationship, dimension=8),
  tf.contrib.layers.embedding_column(race, dimension=8),
  tf.contrib.layers.embedding_column(native_country, dimension=8),
  tf.contrib.layers.embedding_column(occupation, dimension=8),
  age,
  education_num,
  capital_gain,
  capital_loss,
  hours_per_week,
]

print('wide and deep columns configured')

wide and deep columns configured


# Create the model

You can train either a "wide" model, a "deep" model, or a "wide and deep" model, using the classifiers below. Try each one and see what kind of results you get.

* **Wide**: Linear Classifier
* **Deep**: Deep Neural Net Classifier
* **Wide & Deep**: Combined Linear and Deep Classifier

The `hidden_units` or `dnn_hidden_units` argument is to specify the size of each layer of the deep portion of the network. For example, `[12, 20, 15]` would create a network with the first layer of size 12, the second layer of size 20, and a third layer of size 15.

In [72]:
def create_model_dir():
    return 'models/model_' + str(int(time.time()))

# If new_model=False, pass in the desired model_dir 
def get_model(model_type, new_model=True, model_dir=None):
    if new_model or model_dir is None:
        model_dir = create_model_dir() # Comment out this line to continue training a existing model
    print("Model directory = %s" % model_dir)
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = tf.contrib.learn.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = tf.contrib.learn.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[100, 50])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
        m = tf.contrib.learn.DNNLinearCombinedClassifier(
            model_dir=model_dir,
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[100, 70, 50, 25])
        
    print('estimator built')
    
    return m
    
m = get_model(model_type = 'WIDE_AND_DEEP')

Model directory = models/model_1489448804
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121c66710>, '_task_id': 0, '_environment': 'local', '_master': '', '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_is_chief': True, '_save_summary_steps': 100}
estimator built


# Fit the model (train it)

Run `fit()` to train the model. You can experiment with the `train_steps` and `BATCH_SIZE` parameters.

This can take some time, depending on the values chosen for `train_steps` and `BATCH_SIZE`.

Our datafile is hosted on Google Cloud Storage; the reader we created at the beginning knows how to read from it.

In [73]:
train_file = "gs://cloudml-public/census/data/adult.data.csv"
test_file  = "gs://cloudml-public/census/data/adult.test.csv"

train_steps = 1000

m.fit(input_fn=generate_input_fn(train_file, BATCH_SIZE), steps=train_steps)

print('fit done')

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histog

# Evaluate the accuracy of the model
Let's see how the model did. We will evaluate all the test data.

In [76]:
results = m.evaluate(input_fn=generate_input_fn(test_file), steps=100)
print('evaluate done')

print('Accuracy: %s' % results['accuracy'])


evaluate done
Accuracy: 0.84375


# Using Experiments to manage the training workflow

In [69]:
from tensorflow.contrib.learn.python.learn import learn_runner

# Using experiment

# output_dir is an arg passed in by the learn_runner.run() call.
def experiment_fn(output_dir):
    
    train_input_fn = generate_input_fn(train_file, BATCH_SIZE)
    eval_input_fn = generate_input_fn(test_file)
    experiment = tf.contrib.learn.Experiment(
        get_model(model_type='WIDE_AND_DEEP', new_model=False, model_dir=output_dir),
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=1000
    )
    return experiment

In [70]:
# Run the experiment

model_dir=create_model_dir()
learn_runner.run(experiment_fn, model_dir)

Model directory = models/model_1489448561
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13dcf2278>, '_task_id': 0, '_environment': 'local', '_master': '', '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_is_chief': True, '_save_summary_steps': 100}
estimator built
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Tensor("DecodeCSV:14", shape=(?,), dtype=string)
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also

({'accuracy': 0.84675002,
  'accuracy/baseline_label_mean': 0.23675001,
  'accuracy/threshold_0.500000_mean': 0.84675002,
  'auc': 0.90377802,
  'global_step': 1000,
  'labels/actual_label_mean': 0.23675001,
  'labels/prediction_mean': 0.22092056,
  'loss': 0.32621756,
  'precision/positive_threshold_0.500000_mean': 0.74925375,
  'recall/positive_threshold_0.500000_mean': 0.53009504},
 [])

# Conclusions

In this Juypter notebook, we have configured, created, and evaluated a Wide & Deep machine learning model, that combines the powers of a Linear Classifier with a Deep Neural Network, using TensorFlow's tf.learn module.

With this working example in your toolbelt, you are ready to explore the wide (and deep) world of machine learning with TensorFlow! Some ideas to help you get going:
* Change the features we used today. Which columns do you think are correlated and should be crossed? Which ones do you think are just adding noise and could be removed to clean up the model?
* Swap in an entirely new dataset! There are many dataset available on the web, or use a dataset you possess! Check out https://archive.ics.uci.edu/ml to find your own dataset. 